# Restaurant Inventory Control System

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import math

class InventoryControlApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Restaurant Inventory Control System")
        
        # Full-screen mode
        self.root.attributes('-fullscreen', True)

        self.root.configure(bg="#1E1E2E")

        self.sales_data = pd.DataFrame()
        self.filtered_data = pd.DataFrame()
        self.predicted_demand = {}
        self.current_stock = 0  

        self.entry_demand = None
        self.entry_ordering_cost = None
        self.entry_holding_cost = None
        self.entry_current_stock = None
        self.entry_safety_stock = None

        self.create_sidebar()
        self.create_main_interface()
        self.create_exit_button()

    def create_sidebar(self):
        self.sidebar = tk.Frame(self.root, bg="#282A36", width=250, height=self.root.winfo_height())
        self.sidebar.pack(side="left", fill="y", padx=10, pady=10)

        title_label = tk.Label(
            self.sidebar, text="Inventory Control", font=("Times New Roman", 16, "bold"), fg="#50FA7B", bg="#282A36"
        )
        title_label.pack(pady=20)

        # Date Entry
        date_frame = tk.Frame(self.sidebar, bg="#282A36")
        date_frame.pack(pady=10, padx=10, fill="x")

        tk.Label(
            date_frame, text="Date (YYYY-MM-DD):", font=("Times New Roman", 10), fg="#F8F8F2", bg="#282A36"
        ).pack(pady=5)
        
        self.entry_date = tk.Entry(date_frame, font=("Times New Roman", 10), width=15)
        self.entry_date.pack(pady=5)
        
        self.add_sidebar_button("Filter Data by Date", self.filter_data_by_date)
        self.add_sidebar_button("Predict Demand", self.predict_demand)
        self.add_sidebar_button("Inventory Report", self.show_inventory_report)

        # EOQ Inputs
        self.add_sidebar_input("Annual Demand (units):", "demand")
        self.add_sidebar_input("Ordering Cost per Order:", "ordering_cost")
        self.add_sidebar_input("Holding Cost per Unit per Year:", "holding_cost")
        self.add_sidebar_button("Calculate EOQ", self.calculate_eoq)
        
        # Stock inputs
        self.add_sidebar_input("Current Stock:", "current_stock")
        self.add_sidebar_input("Safety Stock:", "safety_stock")
        
        self.add_sidebar_button("Replenishment", self.calculate_replenishment)
        self.add_sidebar_button("Re-order Point", self.calculate_reorder_point)
        
        # Food Item Selection
        tk.Label(self.sidebar, text="Select Food Items:", font=("Times New Roman", 10), fg="#F8F8F2", bg="#282A36").pack(pady=5)
        self.listbox_items = tk.Listbox(self.sidebar, selectmode=tk.MULTIPLE, font=("Times New Roman", 10), bg="#44475A", fg="#F8F8F2")
        self.listbox_items.pack(pady=5, padx=10, fill="x")

        # Populate items when data is loaded
        self.populate_item_list()

    def populate_item_list(self):
        self.listbox_items.delete(0, tk.END)
        if not self.sales_data.empty:
            unique_items = self.sales_data['item_name'].unique()
            for item in unique_items:
                self.listbox_items.insert(tk.END, item)

    
    def add_sidebar_button(self, text, command):
        button = tk.Button(
            self.sidebar, text=text, font=("Times New Roman", 12), bg="#44475A", fg="#F8F8F2", bd=0,
            activebackground="#6272A4", activeforeground="#FFFFFF", height=2, command=command
        )
        button.pack(fill="x", pady=5, padx=10)

    def add_sidebar_input(self, label, attr_name):
        frame = tk.Frame(self.sidebar, bg="#282A36")
        frame.pack(pady=5, padx=10, fill="x")
        
        tk.Label(
            frame, text=label, font=("Times New Roman", 10), fg="#F8F8F2", bg="#282A36"
        ).pack(pady=5)

        entry_widget = tk.Entry(frame, font=("Times New Roman", 10), width=15)
        entry_widget.pack(pady=5)

        setattr(self, f"entry_{attr_name}", entry_widget)
    
    def create_main_interface(self):
        # Main Frame: Centered frame for content with a white border
        self.main_frame = tk.Frame(self.root, bg="#1E1E2E", highlightbackground="white", highlightthickness=3)
        self.main_frame.pack(expand=True, fill="both", padx=20, pady=20)
        
        # Scrollable frame for the content
        self.scrollbar = ttk.Scrollbar(self.main_frame, orient="vertical")
        self.scrollbar.pack(side="right", fill="y")
        
        # Center the header text
        header = tk.Label(
            self.main_frame, text="Restaurant Inventory Dashboard", font=("Times New Roman", 20, "bold"), fg="#FF79C6", bg="#1E1E2E"
        )
        header.pack(pady=20, anchor="center")

        # Output labels centered at the top
        self.label_peak_hour = self.create_output_label("Peak Hour Demand: ")
        self.label_inventory_report = self.create_output_label("Inventory Report: ")
        self.label_eoq_result = self.create_output_label("EOQ: ")

        # Graph Frame 
        self.graph_frame = tk.Frame(self.main_frame, bg="#1E1E2E")
        self.graph_frame.pack(pady=10, padx=10, fill="both", expand=True)

        self.figure = plt.Figure(figsize=(10, 5), dpi=100)
        self.ax = self.figure.add_subplot(111)
        self.canvas_graph = FigureCanvasTkAgg(self.figure, master=self.graph_frame)
        self.canvas_graph.get_tk_widget().pack(fill="both", expand=True)
        self.graph_frame.pack_forget()  

       
        self.label_replenishment = self.create_output_label("Replenishment Quantity: ")
        self.label_reorder_point = self.create_output_label("Re-order Point: ")

    def create_output_label(self, text):
        label = tk.Label(
            self.main_frame, text=text, font=("Times New Roman", 12, "bold"), fg="#50FA7B", bg="#1E1E2E", wraplength=1000
        )
        label.pack(pady=10, anchor="center")
        return label

    def create_exit_button(self):
        exit_button = tk.Button(
            self.root, text="Exit", font=("Times New Roman", 12), bg="#44475A", fg="#F8F8F2", bd=0,
            activebackground="#6272A4", activeforeground="#FFFFFF", height=2, command=self.root.destroy
        )
        exit_button.pack(side="bottom", pady=10)

    def load_sales_data(self):
        """Load sales data from the provided CSV file"""
        try:
            self.sales_data = pd.read_csv("updated_restaurant_inventory.csv")
            self.sales_data['timestamp'] = pd.to_datetime(self.sales_data['timestamp'])
            self.update_stock_levels()  
        except Exception as e:
            self.label_inventory_report.config(text=f"Failed to load sales data: {e}")
    
    def update_stock_levels(self):
        """Update current stock levels based on sales data"""
        if not self.sales_data.empty:
            total_sales = self.sales_data['Sales_Quantity'].sum()
            self.current_stock = max(0, self.current_stock - total_sales)
            self.entry_current_stock.delete(0, tk.END)
            self.entry_current_stock.insert(0, str(self.current_stock))

    def filter_data_by_date(self):
        try:
            self.load_sales_data()
            date_str = self.entry_date.get()
            selected_date = datetime.strptime(date_str, "%Y-%m-%d").date()

            # Filter data by date
            self.filtered_data = self.sales_data[
                self.sales_data['timestamp'].dt.date == selected_date
            ]

            # Debug: Print filtered data
            print("Filtered Data:")
            print(self.filtered_data)

            # Update listbox with items from the filtered date
            self.populate_item_list(self.filtered_data)

            if self.filtered_data.empty:
                self.label_inventory_report.config(text=f"No data found for {date_str}.")
            else:
                self.label_inventory_report.config(text=f"Data loaded for {date_str}!")
                self.update_stock_levels()
        except Exception as e:
            self.label_inventory_report.config(text=f"Error: {str(e)}")
        
    def populate_item_list(self, data=None):
        """Populate listbox with items from the provided DataFrame (default: full dataset)"""
        data = data if data is not None else self.sales_data
        self.listbox_items.delete(0, tk.END)
        if not data.empty:
            unique_items = data['item_name'].unique()
            for item in unique_items:
                self.listbox_items.insert(tk.END, item)
        
    def predict_demand(self):
        if self.filtered_data.empty or not self.listbox_items.curselection():
            self.label_inventory_report.config(text="Please select items and filter data first!")
            return

        self.predicted_demand = {}
        selected_items = [self.listbox_items.get(i) for i in self.listbox_items.curselection()]
        peak_hour_demand = {}

        # Print selected items
        print(f"Selected Items: {selected_items}")

        # Plot setup for the main interface (line graph)
        self.ax.clear()

        for item in selected_items:
            item_data = self.filtered_data[self.filtered_data['item_name'] == item]
            item_data['hour'] = item_data['timestamp'].dt.hour
            hourly_sales = item_data.groupby('hour', as_index=False)['Sales_Quantity'].sum()

            # Train model per item
            model = LinearRegression()
            X = hourly_sales['hour'].values.reshape(-1, 1)
            y = hourly_sales['Sales_Quantity'].values
            model.fit(X, y)

            # Predict for 24 hours
            future_hours = np.array(range(24)).reshape(-1, 1)
            predicted = model.predict(future_hours)
            self.predicted_demand[item] = {hour: int(pred) for hour, pred in zip(range(24), predicted)}

            # Print predicted demand for the item
            print(f"Predicted Demand for {item}: {self.predicted_demand[item]}")

            # Find peak hour demand
            peak_hour = np.argmax(predicted)
            peak_hour_demand[item] = (peak_hour, int(predicted[peak_hour]))

            # Plotting in the main interface (line graph)
            self.ax.plot(hourly_sales['hour'], y, label=f"{item} Actual Sales", marker="o")
            self.ax.plot(range(24), predicted, label=f"{item} Predicted Demand", linestyle="--")

        # Customize the main interface graph
        self.ax.set_title("Hourly Sales and Predicted Demand")
        self.ax.set_xlabel("Hour")
        self.ax.set_ylabel("Quantity Sold")
        self.ax.legend()
        self.ax.grid()

        # Draw the main interface graph
        self.canvas_graph.draw()
        self.graph_frame.pack()

        # Display peak hour demand
        peak_hour_text = "Peak Hour Demand:\n"
        for item, (hour, demand) in peak_hour_demand.items():
            peak_hour_text += f"{item}: Hour {hour}, Demand {demand} units\n"
        self.label_peak_hour.config(text=peak_hour_text)

        # Show bar graph in a pop-up window
        self.show_bar_graph(selected_items)
        
    def show_bar_graph(self, selected_items):
        """Show individual bar graphs for each selected item in separate pop-up windows"""
        for item in selected_items:
            # Create a new pop-up window for each item
            popup = tk.Toplevel(self.root)
            popup.title(f"Demand Comparison for {item}")

            fig = plt.Figure(figsize=(10, 5), dpi=100)
            ax = fig.add_subplot(111)

            bar_width = 0.35  # Width of the bars
            opacity = 0.8  # Opacity of the bars
            hours = list(range(24))  # Hours for the X-axis

            # Get data for the current item
            item_data = self.filtered_data[self.filtered_data['item_name'] == item]
            item_data['hour'] = item_data['timestamp'].dt.hour
            hourly_sales = item_data.groupby('hour', as_index=False)['Sales_Quantity'].sum()

            # Plot actual sales
            ax.bar(
                [h - bar_width / 2 for h in hours],  # X positions for actual sales
                hourly_sales['Sales_Quantity'],  # Actual sales data
                width=bar_width,
                alpha=opacity,
                label="Actual Sales"
            )

            # Plot predicted demand
            ax.bar(
                [h + bar_width / 2 for h in hours],  # X positions for predicted demand
                list(self.predicted_demand[item].values()),  # Predicted demand data
                width=bar_width,
                alpha=opacity,
                label="Predicted Demand"
            )

            # Customize the bar graph
            ax.set_title(f"Actual vs Predicted Demand for {item}")
            ax.set_xlabel("Hour")
            ax.set_ylabel("Quantity Sold")
            ax.set_xticks(hours)
            ax.legend()
            ax.grid()

            # Embed the bar graph in the pop-up window
            canvas = FigureCanvasTkAgg(fig, master=popup)
            canvas.draw()
            canvas.get_tk_widget().pack(fill="both", expand=True)
            
    def show_inventory_report(self):
        """Generate and display inventory report"""
        if not self.predicted_demand:
            self.label_inventory_report.config(text="Please predict demand first!")
            return

        # Calculate total sales (food served to customers)
        total_sales = self.filtered_data['Sales_Quantity'].sum()

        # Calculate total predicted demand across all items and hours
        total_predicted_demand = 0
        for item, hourly_demand in self.predicted_demand.items():
            total_predicted_demand += sum(hourly_demand.values())

        # Calculate food waste
        # Assuming initial stock is available in the data
        initial_stock = self.filtered_data['Stock_Level'].iloc[0] if not self.filtered_data.empty else 0

        # Food bought = initial stock + replenishment (if any)
        # For simplicity, assume replenishment is not considered here
        food_bought = initial_stock

        # Food waste = 4% of food bought + 5% of food served to customers
        food_waste = (0.04 * food_bought) + (0.05 * total_sales)

        # Calculate EOQ
        try:
            demand = float(self.entry_demand.get()) if self.entry_demand.get() else total_predicted_demand
            ordering_cost = float(self.entry_ordering_cost.get()) if self.entry_ordering_cost.get() else 1
            holding_cost = float(self.entry_holding_cost.get()) if self.entry_holding_cost.get() else 1

            if demand <= 0 or ordering_cost <= 0 or holding_cost <= 0:
                raise ValueError("Values must be positive.")

            eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)
            eoq_result = f"EOQ: {eoq:.2f} units"
        except ValueError as e:
            eoq_result = f"EOQ calculation failed: {e}"

        # Generate report
        report = (
            f"Inventory Report\n"
            f"----------------\n"
            f"Total Sales (Food Served): {total_sales}\n"
            f"Total Predicted Demand: {total_predicted_demand}\n"
            f"Initial Stock (Food Bought): {initial_stock}\n"
            f"Estimated Food Waste: {food_waste:.2f} units\n"
            f"{eoq_result}\n"
        )

        self.label_inventory_report.config(text=report)
        
    def calculate_eoq(self):
        """Calculate EOQ based on user input"""
        try:
            demand = float(self.entry_demand.get())
            ordering_cost = float(self.entry_ordering_cost.get())
            holding_cost = float(self.entry_holding_cost.get())

            if demand <= 0 or ordering_cost <= 0 or holding_cost <= 0:
                raise ValueError("Values must be positive.")

            eoq = math.sqrt((2 * demand * ordering_cost) / holding_cost)
            self.label_eoq_result.config(text=f"EOQ: {eoq:.2f} units")
        except ValueError as e:
            self.label_eoq_result.config(text=f"Invalid input: {e}")

    def calculate_replenishment(self):
        try:
            safety_stock = float(self.entry_safety_stock.get())
            results = []

            for item in self.predicted_demand:
                # Get latest stock level for the item
                item_stock = self.filtered_data[self.filtered_data['item_name'] == item]['Stock_Level'].iloc[-1]

                lead_time_demand = sum(self.predicted_demand[item].values())
                replenishment = max(0, (safety_stock + lead_time_demand) - item_stock)
                results.append(f"{item}: {replenishment} units")

            self.label_replenishment.config(text="Replenishment:\n" + "\n".join(results))
            self.label_replenishment.pack()
        except Exception as e:
            self.label_replenishment.config(text=f"Error: {str(e)}")

    def calculate_reorder_point(self):
        try:
            safety_stock = float(self.entry_safety_stock.get())
            results = []


            for item in self.predicted_demand:
                lead_time_demand = sum(self.predicted_demand[item].values())
                reorder_point = lead_time_demand + safety_stock
                results.append(f"{item}: {reorder_point} units")
                
            self.label_reorder_point.config(text="Re-order Points:\n" + "\n".join(results))
            self.label_reorder_point.pack()
        except Exception as e:
            self.label_reorder_point.config(text=f"Error: {str(e)}")
            
if __name__ == "__main__":
    root = tk.Tk()
    app = InventoryControlApp(root)
    root.mainloop()
